In [1]:
import numpy as np
def k(n): return str(int(n)/1000) + 'K'
def kb(n):
    if n > 1000000:
        return str(int(100*(n/1000000))/100.0) + 'MB'
    else:
        return str(int(n)/1000) + 'KB'
def msz(p, b): return p*b/8.0
def score(e,b,m): return int(10000*np.log10(b*m*e))/10000.0

def tups_to_latex(lst):
    for tup in lst:
        print(" & ".join([str(x) for x in tup]) + " \\\\ \hline")

Baseline Models

In [2]:
# FCN1: (kernel + bias + batchnorm)
def fcn(error, lsize, bw = 32, bn=True, name="FCN1"):
    n_params = (1000*lsize + lsize) + (lsize*lsize + lsize)*3 + (lsize*255 + 255)
    if bn: n_params += (lsize*2)*4
    mults = (1000*lsize) + (lsize*lsize)*3 + (lsize*255)
    if bn: mults += (lsize*2)*4
    size = msz(n_params,bw)
    tup = name, error, k(mults), k(n_params), kb(size), score(error,size,mults)
    print(tup); return tup

def cnn(error, bw = 32, bn=True, name="CNN"):
    n_params = (5*5*6 + 6) + (240*256 + 256) + (256*256 + 256)*2 + (256*255 + 255)
    if bn: n_params += 6*2 + (256*2)*3
    mults = (1000)*6 + (240*256) + (256*256)*2 + (256*255)
    if bn: mults += (256*2)*3 + 6
    size = msz(n_params,bw)
    tup = name, error, k(mults), k(n_params), kb(size), score(error,size,mults)
    print(tup); return tup
    
def lcn(error, bw = 32, bn=True, name="LCN"):
    n_params = (10*10*1*25 + 25)*10 + (250*256 + 256) + (256*256 + 256)*2 + (256*255 + 255)
    if bn: n_params += (25*2)*10 + (256*2)*3
    mults = (1000)*25 + (250*256) + (256*256)*2 + (256*255)
    if bn: mults += (256*2)*3 + (25*2)*10
    size = msz(n_params,bw)
    tup = name, error, k(mults), k(n_params), kb(size), score(error,size,mults)
    print(tup); return tup
    
def maxout_big(error, bw = 32, bn=True, name="Maxout"):
    n_params = (1000*1000 + 1000) + (500*1000 + 1000)*2 + (500*255 + 255)
    if bn: n_params += (1000*2)*3
    mults = (1000*1000) + (500*1000)*2 + (500*255)
    if bn: mults += (1000*2)*3
    size = msz(n_params,bw)
    tup = name, error, k(mults), k(n_params), kb(size), score(error,size,mults)
    print(tup); return tup

def maxout_small(error, bw = 32, bn=True, name="Maxout"):
    n_params = (1000*1000 + 1000) + (250*1000 + 1000)*3 + (250*255 + 255)
    if bn: n_params += (1000*2)*4
    mults = (1000*1000) + (250*1000)*3 + (250*255)
    if bn: mults += (1000*2)*4
    size = msz(n_params,bw)
    tup = name, error, k(mults), k(n_params), kb(size), score(error,size,mults)
    print(tup); return tup

def dsc_small(error, bw = 32, bn=True, name="Maxout"):
    n_params = 332179 # count from the log baseexps/train_log/sentfiltNone_dsc2_bnTrue_regTrue_noLRSchedule/log.log
    mults = 360552
    size = msz(n_params,bw)
    tup = name, error, k(mults), k(n_params), kb(size), score(error,size,mults)
    print(tup); return tup

def dsc_large(error, bw = 32, bn=True, name="Maxout"):
    n_params = 1221392 # count from the log baseexps/train_log/sentfiltNone_dsc1_bnTrue_regTrue_noLRSchedule/log.log
    mults = 1233352
    size = msz(n_params,bw)
    tup = name, error, k(mults), k(n_params), kb(size), score(error,size,mults)
    print(tup); return tup

tups = []
tups.append(("Model", "Error", "Mults", "Params", "Bytesize", "Score"))
t1 = fcn(0.093959, 256, bw=32, bn=True, name="Fully Connected, Small")
tups.append(t1)

t2 = fcn(0.029713, 512, bw=32, bn=True, name="Fully Connected, Large")
tups.append(t2)

t3 = cnn(0.12574, bw=32, bn=True, name="Convolutional")
tups.append(t3)

t4 = lcn(0.11322, bw=32, bn=True, name="Locally-Connected")
tups.append(t4)


tups.append(maxout_big(0.26322, bw=32, bn=True, name="Maxout, Large"))
tups.append(maxout_small(0.36653, bw=32, bn=True, name="Maxout, Small"))
tups.append(dsc_small(0.29555, bw=32, bn=True, name="Depth-Separable Conv., Small"))
tups.append(dsc_large(0.11144, bw=32, bn=True, name="Depth-Separable Conv., Large"))
print
print
tups_to_latex(tups)

baselines = [t1[-1], t2[-1], t3[-1], t4[-1]]
print(baselines)

('Fully Connected, Small', 0.093959, '519.936K', '521.215K', '2.08MB', 11.0079)
('Fully Connected, Large', 0.029713, '1433.088K', '1435.391K', '5.74MB', 11.3882)
('Convolutional', 0.12574, '265.334K', '260.519K', '1.04MB', 10.5411)
('Locally-Connected', 0.11322, '287.388K', '288.661K', '1.15MB', 10.5748)
('Maxout, Large', 0.26322, '2133.5K', '2136.755K', '8.54MB', 12.6812)
('Maxout, Small', 0.36653, '1821.75K', '1826.005K', '7.3MB', 12.6881)
('Depth-Separable Conv., Small', 0.29555, '360.552K', '332.179K', '1.32MB', 11.151)
('Depth-Separable Conv., Large', 0.11144, '1233.352K', '1221.392K', '4.88MB', 11.827)
Model & Error & Mults & Params & Bytesize & Score \\ \hline
Fully Connected, Small & 0.093959 & 519.936K & 521.215K & 2.08MB & 11.0079 \\ \hline
Fully Connected, Large & 0.029713 & 1433.088K & 1435.391K & 5.74MB & 11.3882 \\ \hline
Convolutional & 0.12574 & 265.334K & 260.519K & 1.04MB & 10.5411 \\ \hline
Locally-Connected & 0.11322 & 287.388K & 288.661K & 1.15MB & 10.5748 \\ \hlin

In [62]:
tups = []
tups.append(("Model", "Error", "Mults", "Params", "Bytesize", "Score", "Baseline"))
tups.append(fcn(0.093962, 256, bw=32, bn=False, name="Fully Connected, Small") + (baselines[0],))
tups.append(fcn(0.029721, 512, bw=32, bn=False, name="Fully Connected, Large") + (baselines[1],))
tups.append(cnn(0.12604, bw=32, bn=False, name="Convolutional") + (baselines[2],))
tups.append(lcn(0.11410, bw=32, bn=False, name="Locally-Connected") + (baselines[3],))
print
tups_to_latex(tups)

('Fully Connected, Small', 0.093962, '517K', '519K', '2.07MB', 11.0045)
('Fully Connected, Large', 0.029721, '1428K', '1431K', '5.72MB', 11.3858)
('Convolutional', 0.12604, '263K', '258K', '1.03MB', 10.537)
('Locally-Connected', 0.1141, '285K', '286K', '1.14MB', 10.572)

Model & Error & Mults & Params & Bytesize & Score & Baseline \\ \hline
Fully Connected, Small & 0.093962 & 517K & 519K & 2.07MB & 11.0045 & 11.0079 \\ \hline
Fully Connected, Large & 0.029721 & 1428K & 1431K & 5.72MB & 11.3858 & 11.3882 \\ \hline
Convolutional & 0.12604 & 263K & 258K & 1.03MB & 10.537 & 10.5411 \\ \hline
Locally-Connected & 0.1141 & 285K & 286K & 1.14MB & 10.572 & 10.5748 \\ \hline


In [66]:
tups = []
tups.append(("Model", "Error", "Mults", "Params", "Bytesize", "Score", "Baseline"))
tups.append(fcn(0.07430, 512, bw=16, bn=False, name="Fully Connected, Large") + (baselines[1],))
tups.append(cnn(0.20401, bw=16, bn=False, name="Convolutional") + (baselines[2],))
print
tups_to_latex(tups)

('Fully Connected, Large', 0.0743, '1428K', '1431K', '2.86MB', 11.4827)
('Convolutional', 0.20401, '263K', '258K', '517KB', 10.4451)

Model & Error & Mults & Params & Bytesize & Score & Baseline \\ \hline
Fully Connected, Large & 0.0743 & 1428K & 1431K & 2.86MB & 11.4827 & 11.3882 \\ \hline
Convolutional & 0.20401 & 263K & 258K & 517KB & 10.4451 & 10.5411 \\ \hline


In [67]:
tups = []
tups.append(("Model", "Error", "Mults", "Params", "Bytesize", "Score", "Baseline"))
tups.append(fcn(0.23138, 256, bw=4, bn=True, name="FCN, Small (4-bit)") + (baselines[0],))
tups.append(fcn(0.22782, 256, bw=8, bn=True, name="FCN, Small (8-bit)") + (baselines[0],))
tups.append(fcn(0.039581, 512, bw=4, bn=True, name="FCN, Large (4-bit)") + (baselines[1],))
tups.append(fcn(0.026033, 512, bw=8, bn=True, name="FCN, Large (8-bit)") + (baselines[1],))
tups.append(cnn(0.1748, bw=4, bn=True, name="CNN (4-bit)") + (baselines[2],))
tups.append(cnn(0.11593, bw=8, bn=True, name="CNN (8-bit)") + (baselines[2],))
tups.append(lcn(0.26496, bw=4, bn=True, name="LCN (4-bit)") + (baselines[3],))
tups.append(lcn(0.24604, bw=8, bn=True, name="LCN (8-bit)") + (baselines[3],))
print
tups_to_latex(tups)

('FCN, Small (4-bit)', 0.23138, '519K', '521K', '260KB', 10.4962)
('FCN, Small (8-bit)', 0.22782, '519K', '521K', '521KB', 10.7905)
('FCN, Large (4-bit)', 0.039581, '1433K', '1435K', '717KB', 10.6096)
('FCN, Large (8-bit)', 0.026033, '1433K', '1435K', '1.43MB', 10.7287)
('CNN (4-bit)', 0.1748, '265K', '260K', '130KB', 9.7811)
('CNN (8-bit)', 0.11593, '265K', '260K', '260KB', 9.9038)
('LCN (4-bit)', 0.26496, '287K', '288K', '144KB', 10.041)
('LCN (8-bit)', 0.24604, '287K', '288K', '288KB', 10.3098)

Model & Error & Mults & Params & Bytesize & Score & Baseline \\ \hline
FCN, Small (4-bit) & 0.23138 & 519K & 521K & 260KB & 10.4962 & 11.0079 \\ \hline
FCN, Small (8-bit) & 0.22782 & 519K & 521K & 521KB & 10.7905 & 11.0079 \\ \hline
FCN, Large (4-bit) & 0.039581 & 1433K & 1435K & 717KB & 10.6096 & 11.3882 \\ \hline
FCN, Large (8-bit) & 0.026033 & 1433K & 1435K & 1.43MB & 10.7287 & 11.3882 \\ \hline
CNN (4-bit) & 0.1748 & 265K & 260K & 130KB & 9.7811 & 10.5411 \\ \hline
CNN (8-bit) & 0.11593 

In [22]:
def none_ttq(name="None"):
    tup = name, '>0.95', '-', '-', '-'
    print(tup); return tup

def fcn_ttq(error, lsize, qends = True, name="FCN1"):
    ttq_params = (lsize*lsize + lsize)*3 # middle layers
    n_params = (lsize*2)*4 # batch norm
    if qends:
        ttq_params += (1000*lsize + lsize) + (lsize*255 + 255)
    else:
        n_params += (1000*lsize + lsize) + (lsize*255 + 255)
    
    mults = (lsize*2)*4 # batchnorm
    mults += (lsize)*3
    if qends:
        mults += lsize + 255
    else:
        mults += (1000*lsize) + (lsize*255)
    
    print(n_params, ttq_params, ttq_params + n_params)
    size = msz(ttq_params, 3)
    size += msz(n_params,bw)
    tup = name, error, k(mults), kb(size), score(error,size,mults)
    print(tup); return tup

def cnn_ttq(error, name="CNN"):
    # assume batchnorm, not tern ends
    ttq_params = (240*256 + 256) + (256*256 + 256)*2
    n_params = (5*5*6 + 6) + (256*255 + 255)
    n_params += 6*2 + (256*2)*3
    
    mults = (1000)*6 + (256)*3 + (256*255)
    mults += (256*2)*3 + 6
    size = msz(ttq_params, 3)
    size += msz(n_params,bw)
    
    tup = name, error, k(mults), kb(size), score(error,size,mults)
    print(tup); return tup

tups = []
tups.append(("Model", "Error", "Mults", "Bytesize", "Score", "Baseline"))
tups.append(fcn_ttq(0.11046, 512, qends=False, name="FCN, Large (32-Bit Ends)") + (baselines[1],))
tups.append(fcn_ttq(0.12884, 512, qends=True, name="FCN, Large (Tern Ends)") + (baselines[1],))
tups.append(cnn_ttq(0.38439, name="CNN (32-Bit Ends)") + (baselines[2],))
tups.append(none_ttq("CNN (Tern Ends)") + (baselines[2],))
tups.append(none_ttq("FCN, Small (Both)") + (baselines[0],))
tups.append(none_ttq("LCN (Both)") + (baselines[3],))

print
tups_to_latex(tups)

647423 787968 1435391
('FCN, Large (32-Bit Ends)', 0.11046, '648.192K', '942.911KB', 10.8293)
4096 1431295 1435391
('FCN, Large (Tern Ends)', 0.12884, '6.399K', '540.831KB', 8.6492)
('CNN (32-Bit Ends)', 0.38439, '73.59K', '139.719KB', 9.5968)
('CNN (Tern Ends)', '>0.95', '-', '-', '-')
('FCN, Small (Both)', '>0.95', '-', '-', '-')
('LCN (Both)', '>0.95', '-', '-', '-')
Model & Error & Mults & Bytesize & Score & Baseline \\ \hline
FCN, Large (32-Bit Ends) & 0.11046 & 648.192K & 942.911KB & 10.8293 & 11.3882 \\ \hline
FCN, Large (Tern Ends) & 0.12884 & 6.399K & 540.831KB & 8.6492 & 11.3882 \\ \hline
CNN (32-Bit Ends) & 0.38439 & 73.59K & 139.719KB & 9.5968 & 10.5411 \\ \hline
CNN (Tern Ends) & >0.95 & - & - & - & 10.5411 \\ \hline
FCN, Small (Both) & >0.95 & - & - & - & 11.0079 \\ \hline
LCN (Both) & >0.95 & - & - & - & 10.5748 \\ \hline


In [3]:
sparsity_levels = [0, 0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.25, 0.5]
fcn_large_4 = [0.04136947, 0.04136947, 0.04422254, 0.04850214, 0.06847361,
       0.10271041, 0.16405136, 0.22824536, 0.67047076, 0.92724679,
       0.99429387, 0.99857347]
cnn_large_4 = [0.16405136, 0.16690442, 0.19258203, 0.22396576, 0.49500713,
       0.7703281 , 0.92439372, 0.96433666, 0.9871612 , 0.9914408 ,
       0.99857347, 1.        ]
fcn_large_8 = [0.02710414, 0.02710414, 0.03281027, 0.0299572 , 0.03708987,
       0.05563481, 0.06847361, 0.10271041, 0.35378031, 0.68901569,
       0.94864479, 0.99001427]
cnn_large_8 = [0.12268188, 0.11697575, 0.11982882, 0.12696148, 0.16262482,
       0.2168331 , 0.29671897, 0.39657632, 0.72610556, 0.90727532,
       0.98858773, 0.99714693]

In [50]:
def sp(sparsity):
    return str(sparsity*100) + '\%'

def fcns(error, sparsity, lsize = 512, bw = 32, bn=True, name="FCN1"):
    n_params = (1000*lsize + lsize) + (lsize*lsize + lsize)*3 + (lsize*255 + 255)
    if bn: n_params += (lsize*2)*4
    
    mults = (1000*lsize) + (lsize*lsize)*3 + (lsize*255)
    if bn: mults += (lsize*2)*4
    mults = mults*(1-sparsity)
    
    size = msz(n_params,bw)
    tup = name, error, sp(sparsity), k(mults), kb(size), score(error,size,mults)
    
    # print(tup);
    return tup

def cnns(error, sparsity, bw = 32, bn=True, name="CNN"):
    n_params = (5*5*6 + 6) + (240*256 + 256) + (256*256 + 256)*2 + (256*255 + 255)
    if bn: n_params += 6*2 + (256*2)*3
    mults = (1000)*6 + (240*256) + (256*256)*2 + (256*255)
    if bn: mults += (256*2)*3 + 6
    mults = mults*(1-sparsity)
        
    size = msz(n_params,bw)
    tup = name, error, sp(sparsity), k(mults), kb(size), score(error,size,mults)
    # print(tup);
    return tup

def fcn_ttqs(error, sparsity, lsize=512, qends = True, name="FCN1"):
    ttq_params = (lsize*lsize + lsize)*3 # middle layers
    n_params = (lsize*2)*4 # batch norm
    if qends:
        ttq_params += (1000*lsize + lsize) + (lsize*255 + 255)
    else:
        n_params += (1000*lsize + lsize) + (lsize*255 + 255)
    
    mults = (lsize*2)*4 # batchnorm
    mults += (lsize)*3
    if qends:
        mults += lsize + 255
    else:
        mults += (1000*lsize) + (lsize*255)
    mults = mults*(1-sparsity)
    
    print(n_params, ttq_params, ttq_params + n_params)
    size = msz(ttq_params, 3)
    size += msz(n_params,bw)
    tup = name, error, sp(sparsity), k(mults), kb(size), score(error,size,mults)
    print(tup); return tup

def cnn_ttqs(error, sparsity, name="CNN"):
    # assume batchnorm, not tern ends
    ttq_params = (240*256 + 256) + (256*256 + 256)*2
    n_params = (5*5*6 + 6) + (256*255 + 255)
    n_params += 6*2 + (256*2)*3
    
    mults = (1000)*6 + (256)*3 + (256*255)
    mults += (256*2)*3 + 6
    mults = mults*(1-sparsity)
    
    size = msz(ttq_params, 3)
    size += msz(n_params,bw)
    
    tup = name, error, sp(sparsity), k(mults), kb(size), score(error,size,mults)
    print(tup); return tup

In [51]:
tups = []
tups.append(("Model", "Error", "Sparsity", "NZ Mults", "Bytesize", "Score", "Baseline"))

for errs,name,bw in [(fcn_large_4,"FCN, Large (4-bit)",4), (fcn_large_8,"FCN, Large (8-bit)",8),(cnn_large_4,"CNN (4-bit)",4), (cnn_large_8,"CNN (8-bit)",8)]:
    best = 15; best_tup = None
    for i, prune in enumerate(sparsity_levels):
        if 'FCN' in name:
            tup = fcns(round(errs[i], 5), prune, bw=bw, name=name)
        else:
            tup = cnns(round(errs[i], 5), prune, bw=bw, name=name)
        if tup[-1] < best:
            best_tup = tup
            best = tup[-1]
    if 'FCN' in name:
        print(best_tup)
        tups.append(best_tup + (baselines[1],))
    else:
        print(best_tup)
        tups.append(best_tup + (baselines[2],))

t1 = fcn_ttqs(0.12884, 0.22806, name="FCN, Large (TTQ)")
t2 = cnn_ttqs(0.38439, 0.19904, name="CNN (TTQ)")
tups.append(t1 + (baselines[1],))
tups.append(t2 + (baselines[2],))

print
tups_to_latex(tups)

('FCN, Large (4-bit)', 0.04137, '0.1\\%', '1431.654K', '717.695KB', 10.6284)
('FCN, Large (8-bit)', 0.0271, '0.1\\%', '1431.654K', '1.43MB', 10.7457)
('CNN (4-bit)', 0.16405, '0\\%', '265.334K', '130.259KB', 9.7535)
('CNN (8-bit)', 0.11698, '0.1\\%', '265.068K', '260.519KB', 9.9073)
4096 1431295 1435391
('FCN, Large (TTQ)', 0.12884, '22.806\\%', '4.939K', '540.831KB', 8.5368)
('CNN (TTQ)', 0.38439, '19.904\\%', '58.942K', '139.719KB', 9.5004)
Model & Error & Sparsity & NZ Mults & Bytesize & Score & Baseline \\ \hline
FCN, Large (4-bit) & 0.04137 & 0.1\% & 1431.654K & 717.695KB & 10.6284 & 11.3882 \\ \hline
FCN, Large (8-bit) & 0.0271 & 0.1\% & 1431.654K & 1.43MB & 10.7457 & 11.3882 \\ \hline
CNN (4-bit) & 0.16405 & 0\% & 265.334K & 130.259KB & 9.7535 & 10.5411 \\ \hline
CNN (8-bit) & 0.11698 & 0.1\% & 265.068K & 260.519KB & 9.9073 & 10.5411 \\ \hline
FCN, Large (TTQ) & 0.12884 & 22.806\% & 4.939K & 540.831KB & 8.5368 & 11.3882 \\ \hline
CNN (TTQ) & 0.38439 & 19.904\% & 58.942K & 139.71